## Single Table SQL

   ### creating tables in PostgreSQL

acccesing the database: psql -h pg.pg4e.com -p 5432 -U pg4e_cd9300d80a pg4e_cd9300d80a

CREATE TABLE pg4e_debug (
  id SERIAL,
  query VARCHAR(4096),
  result VARCHAR(4096),
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY(id)
);`

In [ ]:
CREATE TABLE pg4e_result (
  id SERIAL,
  link_id INTEGER UNIQUE,
  score FLOAT,
  title VARCHAR(4096),
  note VARCHAR(4096),
  debug_log VARCHAR(8192),
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  updated_at TIMESTAMP
);

In [ ]:
SELECT query, result, created_at FROM pg4e_debug;

##### Result

In [ ]:

 id |                                                     query                                                      |  result   |         created_at         
----+----------------------------------------------------------------------------------------------------------------+-----------+----------------------------
 84 | SELECT id, query, result, created_at FROM pg4e_debug                                                           | Success   | 2021-07-26 12:38:00.263924
 85 | SELECT id, keystr, valstr FROM pg4e_meta                                                                       | Success   | 2021-07-26 12:38:00.278843
 86 | SELECT id, doc FROM docs03 WHERE to_tsquery('english', 'applications') @@ to_tsvector('english', doc);         | Success   | 2021-07-26 12:38:00.28764
 87 | EXPLAIN SELECT id, doc FROM docs03 WHERE to_tsquery('english', 'applications') @@ to_tsvector('english', doc); | Success   | 2021-07-26 12:38:00.290596
 88 | Explain retrieved: Bitmap Heap Scan on docs03  (cost=12.39..81.75 rows=50 width=36)                            | Note only | 2021-07-26 12:38:00.295799
 89 | Explain retrieved:   Recheck Cond: ('''applic'''::tsquery @@ to_tsvector('english'::regconfig, doc))           | Note only | 2021-07-26 12:38:00.305997
 90 | Explain retrieved:   ->  Bitmap Index Scan on fulltext03  (cost=0.00..12.38 rows=50 width=0)                   | Note only | 2021-07-26 12:38:00.31585
 91 | Explain retrieved:         Index Cond: ('''applic'''::tsquery @@ to_tsvector('english'::regconfig, doc))       | Note only | 2021-07-26 12:38:00.328743
(8 rows)

### Entering Many-to-One Data - Automobiles (first application)

In [ ]:
Creating the necessary tables to make the relationships

In [ ]:
CREATE TABLE make (
    id SERIAL,
    name VARCHAR(128) UNIQUE,
    PRIMARY KEY(id)
);

CREATE TABLE model (
  id SERIAL,
  name VARCHAR(128),
  make_id INTEGER REFERENCES make(id) ON DELETE CASCADE,
  PRIMARY KEY(id)
);

#### results

In [ ]:
SELECT make.name, model.name
    FROM model
    JOIN make ON model.make_id = make.id
    ORDER BY make.name LIMIT 5;

In [ ]:
     name     |        name        
--------------+--------------------
 Buick        | Lacrose/Allure
 Buick        | Lacrose/Allure AWD
 Buick        | Lacrose/LeSabre
 Lambhorghini | Urus
 Lambhorghini | Veneno Roadster
(5 rows)

### Entering Many-to-One Data - course roster (second application)

You will normalize the following data (each user gets different data), and insert the following data items into your database, creating and linking all the foreign keys properly. Encode instructor with a role of 1 and a learner with a role of 0.

Data:
Pierce, si106, Instructor
Dimitri, si106, Learner
Evangeline, si106, Learner
Keerah, si106, Learner
Valeria, si106, Learner
Haydyn, si110, Instructor
Arooba, si110, Learner
Grahame, si110, Learner
Nivedita, si110, Learner
Ryan, si110, Learner
Gabriele, si206, Instructor
Kira, si206, Learner
Luic, si206, Learner
Raina, si206, Learner
Somaya, si206, Learner
    


#### creating the tables

In [ ]:
CREATE TABLE student (
    id SERIAL,
    name VARCHAR(128) UNIQUE,
    PRIMARY KEY(id)
);

DROP TABLE course CASCADE;
CREATE TABLE course (
    id SERIAL,
    title VARCHAR(128) UNIQUE,
    PRIMARY KEY(id)
);

DROP TABLE roster CASCADE;
CREATE TABLE roster (
    id SERIAL,
    student_id INTEGER REFERENCES student(id) ON DELETE CASCADE,
    course_id INTEGER REFERENCES course(id) ON DELETE CASCADE,
    role INTEGER,
    UNIQUE(student_id, course_id),
    PRIMARY KEY (id)
);


#### normalizing data and querying result

In [ ]:
Inserting data to students table
insert into student (name) values ('Pierce');
insert into student (name) values ('Dimitri');
insert into student (name) values ('Evangeline');
insert into student (name) values ('Keerah');
insert into student (name) values ('Valeria');
insert into student (name) values ('Haydyn');
insert into student (name) values ('Arooba');
insert into student (name) values ('Grahame');
insert into student (name) values ('Nivedita');
insert into student (name) values ('Ryan');
insert into student (name) values ('Gabriele');
insert into student (name) values ('Kira');
insert into student (name) values ('Luic');
insert into student (name) values ('Raina');
insert into student (name) values ('Somaya');

Inserting data into course table 
insert into course (title) values ('Si106');
insert into course (title) values ('Si110');
insert into course (title) values ('Si206');

Inserting data into roster table
insert into roster (student_id, course_id, role) values (1,1,1);
insert into roster (student_id, course_id, role) values (2,1,0);
insert into roster (student_id, course_id, role) values (3,1,0);
insert into roster (student_id, course_id, role) values (4,1,0);
insert into roster (student_id, course_id, role) values (5,1,0);
insert into roster (student_id, course_id, role) values (6,2,1);
insert into roster (student_id, course_id, role) values (7,2,0);
insert into roster (student_id, course_id, role) values (8,2,0);
insert into roster (student_id, course_id, role) values (9,2,0);
insert into roster (student_id, course_id, role) values (10,2,0);
insert into roster (student_id, course_id, role) values (11,3,1);
insert into roster (student_id, course_id, role) values (12,3,0);
insert into roster (student_id, course_id, role) values (13,3,0);
insert into roster (student_id, course_id, role) values (14,3,0);
insert into roster (student_id, course_id, role) values (15,3,0);


#### query to test out results of data inserted

In [ ]:
SELECT student.name, course.title, roster.role
    FROM student 
    JOIN roster ON student.id = roster.student_id
    JOIN course ON roster.course_id = course.id
    ORDER BY course.title, roster.role DESC, student.name;

#### result

In [ ]:
    name    | title | role 
------------+-------+------
 Pierce     | Si106 |    1
 Dimitri    | Si106 |    0
 Evangeline | Si106 |    0
 Keerah     | Si106 |    0
 Valeria    | Si106 |    0
 Haydyn     | Si110 |    1
 Arooba     | Si110 |    0
 Grahame    | Si110 |    0
 Nivedita   | Si110 |    0
 Ryan       | Si110 |    0
 Gabriele   | Si206 |    1
 Kira       | Si206 |    0
 Luic       | Si206 |    0
 Raina      | Si206 |    0
 Somaya     | Si206 |    0
(15 rows)

### Implementing SQL techniques

#### Tracks database

#### creating the tables

In [ ]:
CREATE TABLE album (
  id SERIAL,
  title VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

CREATE TABLE track (
    id SERIAL,
    title VARCHAR(128),
    len INTEGER, rating INTEGER, count INTEGER,
    album_id INTEGER REFERENCES album(id) ON DELETE CASCADE,
    UNIQUE(title, album_id),
    PRIMARY KEY(id)
);

DROP TABLE IF EXISTS track_raw;
CREATE TABLE track_raw
 (title TEXT, artist TEXT, album TEXT, album_id INTEGER,
  count INTEGER, rating INTEGER, len INTEGER);

Importing data and inserting it to track_raw table

In [ ]:
 \copy track_raw(title,artist,album,count,rating,len) FROM 'library.csv' WITH DELIMITER ',' CSV;
COPY 296

Inserting data into albums table

In [ ]:
insert into album (title) select distinct album from track_raw;                                                         
INSERT 0 41

Inserting data into track table

In [ ]:
insert into track (title, len, rating, count, album_id)
select tr.title, tr.len, tr.rating, tr.count, al.id
from track_raw as tr
join album as al on al.title = tr.album;

#### checking results from inserting

In [ ]:
SELECT track.title, album.title
    FROM track
    JOIN album ON track.album_id = album.id
    ORDER BY track.title LIMIT 3;

In [ ]:
          title            |     album     | id 
----------------------------+---------------+----
 Another One Bites The Dust | Greatest Hits |  5
 Asche Zu Asche             | Herzeleid     | 16
 Beauty School Dropout      | Grease        | 12
(3 rows)

### Manipulating text in PostgreSQL

#### Creating a stored procedure (creating a function that stores the date that it is made and can be updated)

In [ ]:
CREATE TABLE keyvalue ( 
  id SERIAL,
  key VARCHAR(128) UNIQUE,
  value VARCHAR(128) UNIQUE,
  created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
  updated_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
  PRIMARY KEY(id)
);

CREATE OR REPLACE FUNCTION trigger_set_timestamp()
RETURNS TRIGGER as $$
BEGIN
	NEW.updated_at = NOW();
	Return new;
end
$$ language plpgsql;

create trigger set_timestamp
before update on keyvalue
for each row
execute procedure trigger_set_timestamp();


#### result

select * from keyvalue;

key   | value |          created_at           |          updated_at          
----+--------+-------+-------------------------------+------------------------------
  2 | number | 43    | 2021-07-24 00:03:45.452044+00 | 2021-07-24 00:03:45.47357+00

### Using GROUP BY

Creating Northwind Table

In [ ]:
DROP TABLE IF EXISTS suppliers CASCADE;
DROP TABLE IF EXISTS categories CASCADE;
DROP TABLE IF EXISTS customers CASCADE;
DROP TABLE IF EXISTS regions CASCADE;
DROP TABLE IF EXISTS shippers CASCADE;
DROP TABLE IF EXISTS territories CASCADE;
DROP TABLE IF EXISTS employee_territories CASCADE;
DROP TABLE IF EXISTS employees CASCADE;
DROP TABLE IF EXISTS products CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS order_details CASCADE; 


--------------------------------------------------------------------------------
-- Name: suppliers; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------

CREATE TABLE suppliers (
    supp_id smallint NOT NULL PRIMARY KEY,
    company_name varchar(60) NOT NULL,
    contact_name varchar(30),
    contact_title varchar(30),
    address varchar(60),
    city varchar(15),
    region varchar(15),
    postal_code varchar(10),
    country varchar(15),
    phone varchar(24),
    fax varchar(24),
    homepage text
);

--------------------------------------------------------------------------------
-- Name: product_categories; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE categories (
    category_id smallint NOT NULL PRIMARY KEY,
    category_name varchar(15) NOT NULL,
    description text,
    picture bytea
);


--------------------------------------------------------------------------------
-- Name: customers; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------

CREATE TABLE customers (
    cust_id bpchar NOT NULL PRIMARY KEY,
    company_name varchar(40) NOT NULL,
    contact_name varchar(30),
    contact_title varchar(30),
    address varchar(60),
    city varchar(15),
    region varchar(15),
    postal_code varchar(10),
    country varchar(15),
    phone varchar(24),
    fax varchar(24)
);

--------------------------------------------------------------------------------
-- Name: regions; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE regions (
    region_id smallint NOT NULL PRIMARY KEY,
    region_description varchar NOT NULL
);


--------------------------------------------------------------------------------
-- Name: shippers; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE shippers (
    shipper_id smallint NOT NULL PRIMARY KEY,
    company_name varchar(40) NOT NULL,
    phone varchar(24)
);

--------------------------------------------------------------------------------
-- Name: territories; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------

CREATE TABLE territories (
    territory_id character varying(20) NOT NULL PRIMARY KEY,
    territory_description bpchar NOT NULL,
    region_id smallint NOT NULL REFERENCES regions(region_id)
);


--------------------------------------------------------------------------------
-- Name: employee_territories; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE employee_territories (
    emp_id smallint NOT NULL,
    territory_id varchar(20) NOT NULL REFERENCES territories(territory_id),
	PRIMARY KEY (emp_id, territory_id)
);

--------------------------------------------------------------------------------
-- Name: employees; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE employees (
    emp_id smallint NOT NULL PRIMARY KEY,
    last_name character varying(20) NOT NULL,
    first_name character varying(10) NOT NULL,
    title character varying(30),
    title_of_courtesy character varying(25),
    birth_date date,
    hire_date date,
    emp_curr_salary real,
    address character varying(60),
    city character varying(15),
    region character varying(15),
    postal_code character varying(10),
    country character varying(15),
    home_phone character varying(24),
    extension character varying(4),
    photo bytea,
    notes text,
    reports_to smallint,
    photo_path character varying(255)
);

--------------------------------------------------------------------------------
-- Name: products; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------

CREATE TABLE products (
    prod_id smallint NOT NULL PRIMARY KEY,
    prod_name character varying(40) NOT NULL,
    supp_id smallint REFERENCES suppliers(supp_id),
    category_id smallint REFERENCES categories(category_id),
    quantity_per_unit character varying(20),
    unit_price real,
    units_in_stock smallint,
    units_on_order smallint,
    reorder_level smallint,
    discontinued integer NOT NULL
);

--------------------------------------------------------------------------------
-- Name: orders; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE orders (
    order_id smallint NOT NULL PRIMARY KEY,
    cust_id bpchar REFERENCES customers(cust_id),
    emp_id smallint REFERENCES employees(emp_id),
    order_date date,
    required_date date,
    shipped_date date,
    ship_via smallint REFERENCES shippers (shipper_id),
    freight real,
    ship_name character varying(40),
    ship_address character varying(60),
    ship_city character varying(15),
    ship_region character varying(15),
    ship_postal_code character varying(10),
    ship_country character varying(15)
);

--------------------------------------------------------------------------------
-- Name: order_details; Type: TABLE; Schema: public; Owner: -; Tablespace: 
--------------------------------------------------------------------------------
CREATE TABLE order_details (
    order_id smallint NOT NULL REFERENCES orders(order_id),
    prod_id smallint NOT NULL REFERENCES products(prod_id),
    unit_price real NOT NULL,
    quantity smallint NOT NULL,
    discount real NOT NULL,
	PRIMARY KEY (order_id, prod_id)
);

--------------------------------------------------------------------------------
-- add remaining constraints
--------------------------------------------------------------------------------
ALTER TABLE employees 
ADD CONSTRAINT employees_emp_id_fkey FOREIGN KEY (emp_id) REFERENCES employees (emp_id);


#### Using Group by


In [ ]:
SELECT cat.category_name, count(prod_name)
from products prod
inner join categories cat 
on prod.category_id = cat.category_id
group by category_name;


Result:
    
 category_name  | count
----------------+-------
 Grains/Cereals |     7
 Beverages      |    12
 Dairy Products |    10
 Seafood        |    12
 Produce        |     5
 Condiments     |    12
 Meat/Poultry   |     6
 Confections    |    13
(8 rows)


### Using Subquerries

In [ ]:
select cust_id, sum(order_amount) as total_order_amount
	from (select ord.order_id, cust_id, emp.last_name, emp.first_name, prod_id,
	unit_price, quantity, unit_price*quantity as Order_Amount
	from orders ord
	inner join order_details od on ord.order_id = od.order_id
	inner join employees emp on ord.emp_id= emp.emp_id) as emp_orders
group by cust_id;

result:
    
 cust_id | total_order_amount
---------+--------------------
 TOMSP   |  4954.000060558319
 LONEP   |  4258.599994659424
 OLDWO   |  16325.15002822876
 WARTH   |  16617.10011959076
 MAGAA   |  7603.849960327148
 QUEEN   |  30226.10017967224
 VINET   | 1480.0000019073486
 ANTON   |  7515.349945068359
 MORGK   |  5042.200035095215
 GOURL   |  8702.229988098145
 WOLZA   |  3531.949995994568
 GALED   |  836.6999950408936
 RATTC   |  52245.90034675598
 WELLI   |   6480.69997215271
 WILMK   |  3161.349983215332
 FURIB   |  7151.550024986267
 DUMON   | 1615.8999881744385
 FRANK   |  28722.70993900299
 FAMIA   |  4438.899929523468
 TORTU   | 10812.150033950806
 FRANS   | 1545.6999979019165
 PRINI   |  5317.100019454956
 RANCH   |  2844.099998474121
 PERIC   |  4242.200015068054
 CONSH   |  1719.099988937378
 ISLAT   |  6146.299984931946
 LEHMS   | 21282.019976615906
-- More  --

#### Using Subquerry on Select statement

In [ ]:
 select cust.cust_id, (select sum(freight) 
 						from orders ord
					   where cust.cust_id = ord.cust_id) 
from customers cust;

result:

 cust_id |    sum
---------+-----------
 ALFKI   |    225.58
 ANATR   |     97.42
 ANTON   |    268.52
 AROUT   | 471.94998
 BERGS   | 1559.5199
 BLAUS   |    168.26
 BLONP   |    623.66
 BOLID   |    191.17
 BONAP   | 1357.8701
 BOTTM   |  793.9501
 BSBEV   |    281.31
 CACTU   |     72.76
 CENTC   |      3.25
 CHOPS   |    367.24
 COMMI   |    187.82
 CONSH   | 53.620003
 DRACD   |    306.04
 DUMON   | 63.699997
 EASTC   | 832.33997
 ERNSH   |   6205.39
 FAMIA   |    232.75
 FISSA   |
 FOLIG   | 637.93994
 FOLKO   | 1678.0802
 FRANK   | 1403.4398
 FRANR   |    171.42
 FRANS   | 75.130005

#### Computing running Sum on SQL

In [ ]:
select order_id, order_date, freight,
	sum(freight) over (order by order_id) as running_freight_total
from orders;

result:
    
 order_id | order_date | freight | running_freight_total
----------+------------+---------+-----------------------
    10248 | 1996-07-04 |   32.38 |                 32.38
    10249 | 1996-07-05 |   11.61 |                 43.99
    10250 | 1996-07-08 |   65.83 |             109.82001
    10251 | 1996-07-08 |   41.34 |                151.16
    10252 | 1996-07-09 |    51.3 |                202.46
    10253 | 1996-07-10 |   58.17 |                260.63
    10254 | 1996-07-11 |   22.98 |             283.61002
    10255 | 1996-07-12 |  148.33 |                431.94
    10256 | 1996-07-15 |   13.97 |                445.91
    10257 | 1996-07-16 |   81.91 |                527.82
    10258 | 1996-07-17 |  140.51 |                668.33
    10259 | 1996-07-18 |    3.25 |                671.58
    10260 | 1996-07-19 |   55.09 |             726.67004
    10261 | 1996-07-19 |    3.05 |             729.72003
    10262 | 1996-07-22 |   48.29 |                778.01
    10263 | 1996-07-23 |  146.06 |                924.07
    10264 | 1996-07-24 |    3.67 |                927.74
    10265 | 1996-07-25 |   55.28 |                983.02
    10266 | 1996-07-26 |   25.73 |               1008.75
    10267 | 1996-07-29 |  208.58 |               1217.33
    10268 | 1996-07-30 |   66.29 |               1283.62
    10269 | 1996-07-31 |    4.56 |               1288.18
    10270 | 1996-08-01 |  136.54 |             1424.7201
    10271 | 1996-08-01 |    4.54 |             1429.2601
    10272 | 1996-08-02 |   98.03 |             1527.2902
    10273 | 1996-08-05 |   76.07 |             1603.3601
    10274 | 1996-08-06 |    6.01 |             1609.3701

##### running sum with groupings

In [ ]:
select order_id, order_date, cust_id, freight,
	sum(freight) over (partition by cust_id order by order_id) as running_freight_total
from orders;
 order_id | order_date | cust_id | freight | running_freight_total
----------+------------+---------+---------+-----------------------
    10643 | 1997-08-25 | ALFKI   |   29.46 |                 29.46
    10692 | 1997-10-03 | ALFKI   |   61.02 |             90.479996
    10702 | 1997-10-13 | ALFKI   |   23.94 |                114.42
    10835 | 1998-01-15 | ALFKI   |   69.53 |                183.95
    10952 | 1998-03-16 | ALFKI   |   40.42 |                224.37
    11011 | 1998-04-09 | ALFKI   |    1.21 |                225.58
    10308 | 1996-09-18 | ANATR   |    1.61 |                  1.61
    10625 | 1997-08-08 | ANATR   |    43.9 |             45.510002
    10759 | 1997-11-28 | ANATR   |   11.99 |                  57.5
    10926 | 1998-03-04 | ANATR   |   39.92 |                 97.42
    10365 | 1996-11-27 | ANTON   |      22 |                    22
    10507 | 1997-04-15 | ANTON   |   47.45 |                 69.45
    10535 | 1997-05-13 | ANTON   |   15.64 |                 85.09
    10573 | 1997-06-19 | ANTON   |   84.84 |                169.93
    10677 | 1997-09-22 | ANTON   |    4.03 |             173.95999
    10682 | 1997-09-25 | ANTON   |   36.13 |                210.09
    10856 | 1998-01-28 | ANTON   |   58.43 |                268.52
    10355 | 1996-11-15 | AROUT   |   41.95 |                 41.95
    10383 | 1996-12-16 | AROUT   |   34.24 |                 76.19
    10453 | 1997-02-21 | AROUT   |   25.36 |                101.55
    10558 | 1997-06-04 | AROUT   |   72.97 |                174.52
    10707 | 1997-10-16 | AROUT   |   21.74 |             196.26001
    10741 | 1997-11-14 | AROUT   |   10.96 |             207.22002
    10743 | 1997-11-17 | AROUT   |   23.72 |             230.94002
    10768 | 1997-12-08 | AROUT   |  146.32 |                377.26
    10793 | 1997-12-24 | AROUT   |    4.52 |                381.78
    10864 | 1998-02-02 | AROUT   |    3.04 |                384.82
    10920 | 1998-03-03 | AROUT   |   29.61 |                414.43
    10953 | 1998-03-16 | AROUT   |   23.72 |                438.15
    11016 | 1998-04-10 | AROUT   |    33.8 |             471.94998

#### Computing average total

In [ ]:
Computing average total 

select order_id, order_date, cust_id, freight,
	sum(freight) over (order by order_id) as running_freight_total,
	avg(freight) over (order by order_id) as running_freight_average
from orders;


result:
    
 order_id | order_date | cust_id | freight | running_freight_total | running_freight_average
----------+------------+---------+---------+-----------------------+-------------------------
    10248 | 1996-07-04 | VINET   |   32.38 |                 32.38 |      32.380001068115234
    10249 | 1996-07-05 | TOMSP   |   11.61 |                 43.99 |       21.99500036239624
    10250 | 1996-07-08 | HANAR   |   65.83 |             109.82001 |       36.60666751861572
    10251 | 1996-07-08 | VICTE   |   41.34 |                151.16 |      37.790000677108765
    10252 | 1996-07-09 | SUPRD   |    51.3 |                202.46 |       40.49200038909912
    10253 | 1996-07-10 | HANAR   |   58.17 |                260.63 |       43.43833335240682
    10254 | 1996-07-11 | CHOPS   |   22.98 |             283.61002 |       40.51571423666818
    10255 | 1996-07-12 | RICSU   |  148.33 |                431.94 |       53.99250018596649
    10256 | 1996-07-15 | WELLI   |   13.97 |                445.91 |       49.54555575052897
    10257 | 1996-07-16 | HILAA   |   81.91 |                527.82 |       52.78200054168701
    10258 | 1996-07-17 | ERNSH   |  140.51 |                668.33 |      60.757272720336914
    10259 | 1996-07-18 | CENTC   |    3.25 |                671.58 |      55.964999993642174
    10260 | 1996-07-19 | OTTIK   |   55.09 |             726.67004 |       55.89769231356107
    10261 | 1996-07-19 | QUEDE   |    3.05 |             729.72003 |       52.12285714490073

#### Using Row Number

In [ ]:
select prod_id, prod_name, supp_id, category_id,
	row_number () over (order by prod_id)
from products;

result:

prod_id |            prod_name             | supp_id | category_id | row_number
---------+----------------------------------+---------+-------------+------------
       1 | Chai                             |       8 |           1 |          1
       2 | Chang                            |       1 |           1 |          2
       3 | Aniseed Syrup                    |       1 |           2 |          3
       4 | Chef Anton's Cajun Seasoning     |       2 |           2 |          4
       5 | Chef Anton's Gumbo Mix           |       2 |           2 |          5
       6 | Grandma's Boysenberry Spread     |       3 |           2 |          6
       7 | Uncle Bob's Organic Dried Pears  |       3 |           7 |          7
       8 | Northwoods Cranberry Sauce       |       3 |           2 |          8
       9 | Mishi Kobe Niku                  |       4 |           6 |          9
      10 | Ikura                            |       4 |           8 |         10
        
        
With Partition:
    
select prod_id, prod_name, supp_id, category_id,
	row_number () over (partition by supp_id order by prod_id)
from products;

result:

prod_id |            prod_name             | supp_id | category_id | row_number
---------+----------------------------------+---------+-------------+------------
       2 | Chang                            |       1 |           1 |          1
       3 | Aniseed Syrup                    |       1 |           2 |          2
       4 | Chef Anton's Cajun Seasoning     |       2 |           2 |          1
       5 | Chef Anton's Gumbo Mix           |       2 |           2 |          2
      65 | Louisiana Fiery Hot Pepper Sauce |       2 |           2 |          3
      66 | Louisiana Hot Spiced Okra        |       2 |           2 |          4
       6 | Grandma's Boysenberry Spread     |       3 |           2 |          1
       7 | Uncle Bob's Organic Dried Pears  |       3 |           7 |          2
       8 | Northwoods Cranberry Sauce       |       3 |           2 |          3
    
    
Getting the nth row

select prod_id, prod_name, supp_id, category_id, unit_price 
from products
where unit_price = (
	select  unit_price
	from (
		select unit_price, row_number()
		 over (order by unit_price desc) nth
		from (select distinct (unit_price) from products) prices
	) sorted_prices
	where nth =6);

Result:
    
 prod_id |      prod_name       | supp_id | category_id | unit_price
---------+----------------------+---------+-------------+------------
      59 | Raclette Courdavault |      28 |           4 |         55
(1 row)

#### Using rank

In [ ]:
Result:

select prod_id, prod_name, supp_id, category_id, unit_price,
	rank() over (order by unit_price) unit_price_rank
from products;    

prod_id |            prod_name             | supp_id | category_id | unit_price | unit_price_rank
---------+----------------------------------+---------+-------------+------------+-----------------
      33 | Geitost                          |      15 |           4 |        2.5 |               1
      24 | Guaranß Fantßstica               |      10 |           1 |        4.5 |               2
      13 | Konbu                            |       6 |           8 |          6 |               3
      52 | Filo Mix                         |      24 |           5 |          7 |               4
      54 | TourtiΦre                        |      25 |           6 |       7.45 |               5
      75 | Rh÷nbrΣu Klosterbier             |      12 |           1 |       7.75 |               6
      23 | Tunnbr÷d                         |       9 |           5 |          9 |               7
      19 | Teatime Chocolate Biscuits       |       8 |           3 |        9.2 |               8
      47 | Zaanse koeken                    |      22 |           3 |        9.5 |               9
      45 | Rogede sild                      |      21 |           8 |        9.5 |               9
      41 | Jack's New England Clam Chowder  |      19 |           8 |       9.65 |              11
      74 | Longlife Tofu                    |       4 |           7 |         10 |              12
       3 | Aniseed Syrup                    |       1 |           2 |         10 |              12

#### Using Dense Rank

In [ ]:
select prod_id, prod_name, supp_id, category_id, unit_price,
	dense_rank() over (partition by category_id order by unit_price) unit_price_rank
from products;

Results:
    
prod_id |            prod_name             | supp_id | category_id | unit_price | unit_price_rank
---------+----------------------------------+---------+-------------+------------+-----------------
      24 | Guaranß Fantßstica               |      10 |           1 |        4.5 |               1
      75 | Rh÷nbrΣu Klosterbier             |      12 |           1 |       7.75 |               2
      34 | Sasquatch Ale                    |      16 |           1 |         14 |               3
      67 | Laughing Lumberjack Lager        |      16 |           1 |         14 |               3
      70 | Outback Lager                    |       7 |           1 |         15 |               4
      35 | Steeleye Stout                   |      16 |           1 |         18 |               5
      76 | Lakkalik÷÷ri                     |      23 |           1 |         18 |               5
       1 | Chai                             |       8 |           1 |         18 |               5
        
        
        
Using Dense Rank with CTE

with cte as (select prod_id, prod_name, supp_id, category_id, unit_price,
	dense_rank() over (partition by category_id order by unit_price desc) unit_price_rank
from products)

select prod_id, prod_name, unit_price
from cte
where unit_price_rank =1;

### First Value Window Function

In [ ]:
select prod_id, prod_name, supp_id, category_id, unit_price,
	first_value (unit_price) over (order by unit_price) as lowest_price
from products;

Result:
 prod_id |            prod_name             | supp_id | category_id | unit_price | lowest_price
---------+----------------------------------+---------+-------------+------------+--------------
      33 | Geitost                          |      15 |           4 |        2.5 |          2.5
      24 | Guaranß Fantßstica               |      10 |           1 |        4.5 |          2.5
      13 | Konbu                            |       6 |           8 |          6 |          2.5
      52 | Filo Mix                         |      24 |           5 |          7 |          2.5
      54 | TourtiΦre                        |      25 |           6 |       7.45 |          2.5
      75 | Rh÷nbrΣu Klosterbier             |      12 |           1 |       7.75 |          2.5
      23 | Tunnbr÷d                         |       9 |           5 |          9 |          2.5
      19 | Teatime Chocolate Biscuits       |       8 |           3 |        9.2 |          2.5
      47 | Zaanse koeken                    |      22 |           3 |        9.5 |          2.5
    

With Partition:
    
select prod_id, prod_name, category_id, unit_price,
	first_value (prod_name) over (partition by category_id order by unit_price) 
	as lowest_price_category
from products;


#### Using Lag_function and CTE to get variance

In [2]:
with cte as (select shipped_date, sum(freight) as freight
			 from orders
			 group by shipped_date
			 order by shipped_date),
cte2 as	(select shipped_date, freight, 
		lag(freight,1) over (order by shipped_date) as previous_freight
		from cte)

select shipped_date, freight, previous_freight, (freight-previous_freight) as variance 
from cte2;   
 
Result:    
    
shipped_date |  freight   | previous_freight |  variance
--------------+------------+------------------+-------------
 1996-07-10   |      11.61 |                  |
 1996-07-11   |       51.3 |            11.61 |       39.69
 1996-07-12   |      65.83 |             51.3 |   14.530003
 1996-07-15   |     189.67 |            65.83 |      123.84
 1996-07-16   |      90.55 |           189.67 |  -99.119995
 1996-07-17   |      13.97 |            90.55 |      -76.58
 1996-07-22   |      81.91 |            13.97 |       67.94
 1996-07-23   |  163.48999 |            81.91 |    81.57999
 1996-07-25   |      51.54 |        163.48999 |  -111.94999
 1996-07-29   |      55.09 |            51.54 |   3.5499992
 1996-07-30   |       3.05 |            55.09 |      -52.04
 1996-07-31   |     171.79 |             3.05 |   168.73999
 1996-08-02   |  202.82999 |           171.79 |   31.039993
 1996-08-06   |     306.61 |        202.82999 |      103.78
 1996-08-09   |      31.49 |           306.61 |     -275.12
 1996-08-12   |     131.35 |            31.49 |    99.86001
 1996-08-13   |     125.77 |           131.35 |  -5.5800095
 1996-08-14   |      13.84 |           125.77 |  -111.92999
 1996-08-16   |  124.53001 |            13.84 |      110.69
 1996-08-21   |  15.629999 |        124.53001 |  -108.90001
 1996-08-23   |  88.479996 |        15.629999 |       72.85
 1996-08-26   |      76.83 |        88.479996 |  -11.649994
 1996-08-27   |      76.56 |            76.83 | -0.27000427

SyntaxError: invalid syntax (<ipython-input-2-993a83281f82>, line 1)

#### Calculating Sum of Percentage

In [ ]:
with total as (
	select sum(freight) as total_freight
	from orders)

select ship_country, 
	100*(sum(freight)/total.total_freight) as freight_share
	from orders, total
group by ship_country, total.total_freight
order by ship_country;


Result:
    
ship_country |    freight_share
--------------+---------------------
 Argentina    |  0.9217043407261372
 Austria      |  11.381565779447556
 Belgium      |  1.9711825996637344
 Brazil       |   7.514604181051254
 Canada       |   3.384659066796303
 Denmark      |   2.149878442287445
 Finland      |  1.4026048593223095
 France       |   6.525501608848572
 Germany      |  17.374195158481598
 Ireland      |   4.242568090558052
 Italy        |  1.3310804031789303
 Mexico       |  1.7288768664002419
 Norway       | 0.42421985417604446
 Poland       | 0.27060762513428926
 Portugal     |  0.9909190237522125
 Spain        |   1.327153667807579
 Sweden       |   4.985315352678299
 Switzerland  |     2.1072868257761
 UK           |   4.549038410186768
 USA          |  21.205273270606995
 Venezuela    |   4.211679846048355
(21 rows)

### Using Regular Expressions

 create a regular expression to retrieve a subset data from the purpose column of the taxdata table in the readonly database (access details below). Write a regular expressions to retrieve that meet the following criteria:

Lines that have three digit numbers in parenthesis like (567) 

Schema for the tax data table:

readonly=# \d+ taxdata
  Column  |          Type          |
----------+------------------------+
 id       | integer                |
 ein      | integer                |
 name     | character varying(255) |
 year     | integer                |
 revenue  | bigint                 |
 expenses | bigint                 |
 purpose  | text                   |
 ptid     | character varying(255) |
 ptname   | character varying(255) |
 city     | character varying(255) |
 state    | character varying(255) |
 url      | character varying(255) |

In [ ]:
Querying using Regular expression

In [ ]:
SELECT purpose FROM taxdata WHERE purpose ~ '\([0-9][0-9][0-9]\)'ORDER BY purpose DESC LIMIT 3;    

#### Result

In [ ]:
                                                                                                              purpose                                                                                                                                                                                                
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 VNSA WAS ORGANIZED TO PROVIDE FINANCIAL RESOURCES TO OTHER (501) (C)(3) ORGANIZATIONS PRIMARILY THROUGH AN ANNUAL BOOK SALE. BOOKS ARE DONATED.
 VNSA WAS ORGANIZED TO PROVIDE FINANCIAL RESOURCES TO OTHER (501) (C)(3) ORGANIZATIONS PRIMARILY THROUGH AN ANNUAL BOOK SALE. BOOKS ARE DONATED.
 To provide free food distribution to the poor in the North East and surrounding communities Free food distribution to the poor residing in North East and Greenfield townships within the state of Pennsylvania. A total of (293) different households with (720) individuals were served twice per month for a total of 6,512 persons. The total of 130,148 pounds of food was served in 2015.051569
(3 rows)

### Generating Text and concatination

create a table named bigtext with a single TEXT column named content. Insert 100000 records with numbers starting at 100000 and going through 199999 into the table as shown below:


creating table

In [ ]:
CREATE TABLE bigtext (
  content text
);

Generating 100,000 entries

In [ ]:
insert into bigtext (content) Select 'This is record number '|| generate_series(100000,199999) ||' of quite a few text records.';
INSERT 0 100000

### Reverse Index in SQL

 create a table of documents and then produce a reverse index for those documents that identifies each document which contains a particular word using SQL.

creating the tables 

In [ ]:
CREATE TABLE docs01 (id SERIAL, doc TEXT, PRIMARY KEY(id));

CREATE TABLE invert01 (
  keyword TEXT,
  doc_id INTEGER REFERENCES docs01(id) ON DELETE CASCADE
);

inserting data into docs01

In [ ]:
INSERT INTO docs01 (doc) VALUES
('assistants who can take care of many things on our behalf The hardware'),
('in our currentday computers is essentially built to continuously ask us'),
('the question What would you like me to do next'),
('Programmers add an operating system and a set of applications to the'),
('hardware and we end up with a Personal Digital Assistant that is quite'),
('helpful and capable of helping us do many different things'),
('Our computers are fast and have vast amounts of memory and could be very'),
('helpful to us if we only knew the language to speak to explain to the'),
('computer what we would like it to do next If we knew this language'),
('we could tell the computer to do tasks on our behalf that were');

Querries to make invert table`

In [ ]:
select distinct(s.keyword) as keyword, id
from docs01 as D, unnest(string_to_array(lower(D.doc), ' ')) s(keyword)
order by id;

insert into invert01 (keyword, doc_id)
select distinct(s.keyword) as keyword, id
from docs01 as D, unnest(string_to_array(lower(D.doc), ' ')) s(keyword)
order by id;


select distinct id, doc from docs01 as D
join invert01 as I on I.doc_id = D.id
where I.keyword= 'and';


querying result of the table

In [ ]:
SELECT keyword, doc_id FROM invert01 ORDER BY keyword, doc_id LIMIT 10;

keyword    |  doc_id
-----------+--------
a          |    4    
a          |    5    
add        |    4    
amounts    |    7    
an         |    4    
and        |    4    
and        |    5    
and        |    6    
and        |    7    
applications |    4

### Reverse Index (with stop words) in SQL

create a table of documents and then produce a reverse index with stop words for those documents that identifies each document which contains a particular word using SQL.

creating the table for reverse index that looks for stop words.

In [ ]:
CREATE TABLE docs02 (id SERIAL, doc TEXT, PRIMARY KEY(id));

CREATE TABLE invert02 (
  keyword TEXT,
  doc_id INTEGER REFERENCES docs02(id) ON DELETE CASCADE
);

inserting data into table

In [ ]:
INSERT INTO docs02 (doc) VALUES
('assistants who can take care of many things on our behalf The hardware'),
('in our currentday computers is essentially built to continuously ask us'),
('the question What would you like me to do next'),
('Programmers add an operating system and a set of applications to the'),
('hardware and we end up with a Personal Digital Assistant that is quite'),
('helpful and capable of helping us do many different things'),
('Our computers are fast and have vast amounts of memory and could be very'),
('helpful to us if we only knew the language to speak to explain to the'),
('computer what we would like it to do next If we knew this language'),
('we could tell the computer to do tasks on our behalf that were');

creating table with stop words:

In [ ]:
CREATE TABLE stop_words (word TEXT unique);

INSERT INTO stop_words (word) VALUES 
('i'), ('a'), ('about'), ('an'), ('are'), ('as'), ('at'), ('be'), 
('by'), ('com'), ('for'), ('from'), ('how'), ('in'), ('is'), ('it'), ('of'), 
('on'), ('or'), ('that'), ('the'), ('this'), ('to'), ('was'), ('what'), 
('when'), ('where'), ('who'), ('will'), ('with');

executing the query

In [ ]:
insert into invert02 (keyword, doc_id)
select distinct(s.keyword) as keyword, id
from docs02 as D, unnest(string_to_array(lower(D.doc), ' ')) s(keyword)
where s.keyword not in (select word from stop_words)
order by id;  

results

In [ ]:
SELECT keyword, doc_id FROM invert02 ORDER BY keyword, doc_id LIMIT 10;

keyword    |  doc_id
-----------+--------
add        |    4    
amounts    |    7    
and        |    4    
and        |    5    
and        |    6    
and        |    7    
applications |    4    
ask        |    2    
assistant  |    5    
assistants |    1    

### String Array Gin Index

create a table of documents and then produce a GIN-based text[] reverse index for those documents that identifies each document which contains a particular word using SQL.

CREATE TABLE docs03 (id SERIAL, doc TEXT, PRIMARY KEY(id));

CREATE INDEX array03 ON docs03 USING gin(string_to_array(lower(doc), ' ') array_ops);

inserting data

In [ ]:
INSERT INTO docs03 (doc) VALUES
('assistants who can take care of many things on our behalf The hardware'),
('in our currentday computers is essentially built to continuously ask us'),
('the question What would you like me to do next'),
('Programmers add an operating system and a set of applications to the'),
('hardware and we end up with a Personal Digital Assistant that is quite'),
('helpful and capable of helping us do many different things'),
('Our computers are fast and have vast amounts of memory and could be very'),
('helpful to us if we only knew the language to speak to explain to the'),
('computer what we would like it to do next If we knew this language'),
('we could tell the computer to do tasks on our behalf that were');


INSERT INTO docs03 (doc) SELECT 'Neon ' || generate_series(10000,20000);

querying using GIN Index

In [ ]:
SELECT id, doc FROM docs03 WHERE '{applications}' <@ string_to_array(lower(doc), ' ');

result

In [ ]:
 id |                                 doc                                  
----+----------------------------------------------------------------------
  4 | Programmers add an operating system and a set of applications to the
(1 row)

Proving GIN index worked

In [ ]:
EXPLAIN SELECT id, doc FROM docs03 WHERE '{applications}' <@ string_to_array(lower(doc), ' ');

In [ ]:
                             QUERY PLAN                                   
--------------------------------------------------------------------------------
 Seq Scan on docs03  (cost=0.00..230.19 rows=50 width=15)
   Filter: ('{applications}'::text[] <@ string_to_array(lower(doc), ' '::text))
(2 rows)

### GIN ts_vector Index

create a table of documents and then produce a GIN-based ts_vector index on the documents.

creating the table and inserting data 

In [ ]:
CREATE TABLE docs03 (id SERIAL, doc TEXT, PRIMARY KEY(id));

CREATE INDEX fulltext03 ON docs03 USING gin(to_tsvector('english', doc));

INSERT INTO docs03 (doc) VALUES
('assistants who can take care of many things on our behalf The hardware'),
('in our currentday computers is essentially built to continuously ask us'),
('the question What would you like me to do next'),
('Programmers add an operating system and a set of applications to the'),
('hardware and we end up with a Personal Digital Assistant that is quite'),
('helpful and capable of helping us do many different things'),
('Our computers are fast and have vast amounts of memory and could be very'),
('helpful to us if we only knew the language to speak to explain to the'),
('computer what we would like it to do next If we knew this language'),
('we could tell the computer to do tasks on our behalf that were');


INSERT INTO docs03 (doc) SELECT 'Neon ' || generate_series(10000,20000);

seeing if ts_vector works

In [ ]:
 id |                                 doc                               
   
----+-------------------------------------------------------------------
---
  4 | Programmers add an operating system and a set of applications to t
he

In [ ]:
EXPLAIN SELECT id, doc FROM docs03 WHERE to_tsquery('english', 'applications') @@ to_tsvector('english', doc);

In [ ]:
EXPLAIN SELECT id, doc FROM docs03 WHERE to_tsquery('english', 'instructions') @@ to_tsvector('english', doc);
                                       QUERY PLAN
-----------------------------------------------------------------------------------------
 Bitmap Heap Scan on docs03  (cost=208.27..268.71 rows=35 width=36)
   Recheck Cond: ('''instruct'''::tsquery @@ to_tsvector('english'::regconfig, doc))
   ->  Bitmap Index Scan on fulltext03  (cost=0.00..208.26 rows=35 width=0)
         Index Cond: ('''instruct'''::tsquery @@ to_tsvector('english'::regconfig, doc))
(4 rows)

### Using Python with SQL

A web crawler application that parses email data from a file and updates a database

In [ ]:
import sqlite3
import time
import ssl
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urljoin
from urllib.parse import urlparse
import re
from datetime import datetime, timedelta

# Not all systems have this so conditionally define parser
try:
    import dateutil.parser as parser
except:
    pass

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(tdate)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print 'Bad Date:',md
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('content.sqlite')
cur = conn.cursor()

baseurl = "http://mbox.dr-chuck.net/sakai.devel/"

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER UNIQUE, email TEXT, sent_at TEXT,
     subject TEXT, headers TEXT, body TEXT)''')

# Pick up where we left off
start = None
cur.execute('SELECT max(id) FROM Messages' )
try:
    row = cur.fetchone()
    if row is None :
        start = 0
    else:
        start = row[0]
except:
    start = 0

if start is None : start = 0

many = 0
count = 0
fail = 0
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many messages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    start = start + 1
    cur.execute('SELECT id FROM Messages WHERE id=?', (start,) )
    try:
        row = cur.fetchone()
        if row is not None : continue
    except:
        row = None

    many = many - 1
    url = baseurl + str(start) + '/' + str(start + 1)

    text = "None"
    try:
        # Open with a timeout of 30 seconds
        document = urllib.request.urlopen(url, None, 30, context=ctx)
        text = document.read().decode()
        if document.getcode() != 200 :
            print("Error code=",document.getcode(), url)
            break
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page",url)
        print("Error",e)
        fail = fail + 1
        if fail > 5 : break
        continue

    print(url,len(text))
    count = count + 1

    if not text.startswith("From "):
        print(text)
        print("Did not find From ")
        fail = fail + 1
        if fail > 5 : break
        continue

    pos = text.find("\n\n")
    if pos > 0 :
        hdr = text[:pos]
        body = text[pos+2:]
    else:
        print(text)
        print("Could not find break between headers and body")
        fail = fail + 1
        if fail > 5 : break
        continue

    email = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) == 1 :
        email = x[0];
        email = email.strip().lower()
        email = email.replace("<","")
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) == 1 :
            email = x[0];
            email = email.strip().lower()
            email = email.replace("<","")

    date = None
    y = re.findall('\Date: .*, (.*)\n', hdr)
    if len(y) == 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except:
            print(text)
            print("Parse fail",tdate)
            fail = fail + 1
            if fail > 5 : break
            continue

    subject = None
    z = re.findall('\Subject: (.*)\n', hdr)
    if len(z) == 1 : subject = z[0].strip().lower();

    # Reset the fail counter
    fail = 0
    print("   ",email,sent_at,subject)
    cur.execute('''INSERT OR IGNORE INTO Messages (id, email, sent_at, subject, headers, body)
        VALUES ( ?, ?, ?, ?, ?, ? )''', ( start, email, sent_at, subject, hdr, body))
    if count % 50 == 0 : conn.commit()
    if count % 100 == 0 : time.sleep(1)

conn.commit()
cur.close()
